## Case Study: Solve recaptcha v2 with 2captcha service and requests

Created by [tanyongsheng.net](https://tanyongsheng.net)

Reference: 
1. How to Solve Captcha / ReCaptcha - Python and 2captcha https://www.youtube.com/watch?v=R6QddZzCOwM & https://github.com/eupendra/2captcha_demo/blob/main/demo_requests.py
2. Bypass captcha in Python: https://2captcha.com/lang/python

----

Prerequisite:
1. Buy 2Captcha credit at https://2captcha.com/?from=22013304 (Note: this is an affiliate link)

In [3]:
!pip install requests
!pip install lxml
!pip install 2captcha-python

In [ ]:
login_page_url = "https://www.investingnote.com/users/sign_in"

In [2]:
import requests
from lxml import html

# get crsf token
session = requests.Session()
response = session.get(url=login_page_url)

tree = html.fromstring(response.content)
